# Loading margin catalog with Almanac

**Author**: Melissa DeLucchi

This notebook is a proof-of-concept for loading a catalog in lsdb, with its associated margin catalog. This uses catalog path, links, and other metadata stored in the hipscat `Almanac`.


In [1]:
from hipscat.inspection.almanac import Almanac
import lsdb

Environment variables are hard with jupyter (and in shared compute environments), and I'm just coercing the almanac environment variable here.

In [2]:
%env HIPSCAT_ALMANAC_DIR=/data3/epyc/data3/hipscat/almanac

env: HIPSCAT_ALMANAC_DIR=/data3/epyc/data3/hipscat/almanac


What follows is a simple demonstration of a method to fetch the margin catalog of the primary catalog. 

- if the user provides a name of a margin, use that
- if the catalog has a single margin catalog registered, use that
- if there are 0 margins, print a warning
- if there are >1 marings, print a warning with the name of the catalogs to pick from

In [3]:
def load_catalog_and_margin(catalog_name, margin_name=None):
    almanac = Almanac()
    catalog = almanac.get_almanac_info(catalog_name)
    print(catalog.catalog_path)
    if len(catalog.margins) == 1 or margin_name is not None:
        if margin_name is None:
            margin_name = catalog.margins[0].catalog_name
        print("using margin", margin_name)
        margin_path = almanac.get_almanac_info(margin_name).catalog_path
        margin_catalog = lsdb.read_hipscat(margin_path)

        primary_catalog = lsdb.read_hipscat(catalog.catalog_path, margin_cache=margin_catalog)
        return primary_catalog
    
    if len(catalog.margins) == 0:
        print("could not find a margin for", catalog_name)
    elif len(catalog.margins) > 1:
        print("multiple margins - pick one")
        for margin in catalog.margins:
            print("  -", margin.catalog_name)
    return lsdb.read_hipscat(catalog.catalog_path)
    

In [4]:
gaia = load_catalog_and_margin("gaia")

/data3/epyc/data3/hipscat/catalogs/gaia_dr3/gaia
using margin gaia_10arcs


In [5]:
allwise = load_catalog_and_margin("allwise")

/data3/epyc/data3/hipscat/catalogs/allwise/allwise
using margin allwise_10arcs


In [6]:
milliquas = load_catalog_and_margin("dr16q_constant")

/data3/epyc/data3/hipscat/catalogs/dr16q_constant
multiple margins - pick one
  - mq_15arcs
  - mq_5arcs


**Discussion**:

I like the idea of making margin selection painless for users. I think we'd want to go a few steps further in our implementation within lsdb:

```
lsdb.read_hipscat(
    catalog_path:str
    use_margin:
        bool - if True, seek out a single margin in the almanac
             - if False (default?) do nothing
        string - if this is a path, load a margin at the provided path
               - else, use the almanac to find a margin with the indicated name associated with this catalog
        MarginCatalog - a pre-loaded margin_catalog
    **kwargs, etc
```